# Изучаем игровую индустрию

- Автор: *Наталья Мартынова*
- Дата: *28 сентября 2025 г.*

### Цели и задачи проекта

<font color='#777778'>Цель: изучить данные о развитии игровой индустрии в начале XXI века.</font>

    
   <font color='#777778'>Задачи:
> - познакомиться с данными о развитии игровой индустрии,
> - подготовить данные к анализу,
> - сделать несколько выборок, чтобы составить общее представление об играх, выпущенных в исследуемый период, и платформах, на которых они вышли.</font>

### Описание данных

<font color='#777778'>В проекте будут использованы данные датасета `/datasets/new_games.csv` с таким описанием:</font>

> - <font color='#777778'>`Name` — название игры;
> - `Platform` — название платформы;
> - `Year of Release` — год выпуска игры;
> - `Genre` — жанр игры;
> - `NA sales` — продажи в Северной Америке (в миллионах проданных копий);
> - `EU sales` — продажи в Европе (в миллионах проданных копий);
> - `JP sales` — продажи в Японии (в миллионах проданных копий);
> - `Other sales` — продажи в других странах (в миллионах проданных копий);
> - `Critic Score` — оценка критиков (от 0 до 100); 
> - `User Score` — оценка пользователей (от 0 до 10);
> - `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board - эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию).</font>

### Содержимое проекта

<font color='#777778'>1. Загрузка данных и знакомство с ними.
    
<font color='#777778'>2. Проверка ошибок в данных и их предобработка: 
    
> - Названия, или метки, столбцов датафрейма.
> - Типы данных. 
> - Наличие пропусков в данных.
> - Явные и неявные дубликаты в данных.
    
<font color='#777778'>3. Фильтрация данных.</font>
    
<font color='#777778'>4. Категоризация данных.</font>

---

## Загрузка данных и знакомство с ними

In [1]:
# Импортируем библиотеки
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Выгружаем данные из датасета /datasets/new_games.csv в датафрейм df
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

- Познакомьтесь с данными: выведите первые строки и результат метода `info()`.


In [3]:
# Выводим первые строки датафрейма на экран
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Выводим информацию о датафрейме
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB
None


Датасет /datasets/new_games.csv содержит 16956 строк и 11 столбцов, в которых содержится информация о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр.

Изучим типы данных и их корректность:
- **Числовые значения с плавающей запятой (float64).** 4 столбца содержат значения с типом данных `float64`: 
- `Year of Release` - с годом выпуска игры. С одной стороны кажется, что этому столбцу лучше подойдёт тип `datetime64`, но так как у нас имеется только значение года, то можно привести столбец к целочисленному типу `int16` - этого будет достаточно для расчётов, формат записи значений примет более подходящий вид (без запятой), а полная запись даты с днём и месяцем нам тут не понадобится).
- для оставшихся трёх столбцов: `NA sales` (продажи в Северной Америке (в миллионах проданных копий)), `Other sales` (продажи в других странах (в миллионах проданных копий) и `Critic Score` (с оценками критиков (от 0 до 100)) - тип с плавающей точкой подходит, но, учитывая диапазоны значений в этих столбцах, можно изменить тип на `float32` для оптимизации ресурсов.

- **Строковые данные (object).** Семь столбцов имеют тип данных `object`:
- столбец `Name` содержит строковую информацию  (названия игр), и ему тип данных `object` подходит. 
- столбцы `Platform` (названия платформ),  `Genre` (жанры игр) и `Rating` (рейтинг организации ESRB) - также содержат строковую информацию, но их можно рассматривать как категориальные признаки. В этом случае можно использовать тип `category`, чтобы улучшить производительность и оптимизировать память, если набор значений ограничен и заведомо известен.
- столбцам же `EU sales` (продажи в Европе (в миллионах проданных копий)), `JP sales` (продажи в Японии (в миллионах проданных копий)) и `User Score` (оценка пользователей (от 0 до 10)) лучше подойдёт числовой тип с плавающей точкой и, учитывая диапазоны значений в этих столбцах, можно выбрать тип `float32`.

После анализа типов данных видим, что в 10-ти из 11-ти столбцов нужно преобразовать типы данных для удобства дальнейшей работы с данными и оптимизации ресурсов системы.

Изучим пропуски в данных:
Пропуски присутствуют в шести из 11-ти столбцов: `Name`, `Year of Release`, `Genre`, `Critic Score`,  `User Score`, `Rating`. В последнем их довольно значительная доля. Больше всего пропусков в столбце `Critic Score`. Следует далее отдельно проанализировать пропуски.

Что касается соответствия названий столбцов их содержимому - всё соответствует и корректно для дальнейшей работы с этими данными. Однако основным стилем написания для Python является snake_case - поэтому для унифицирования стиля нужно будет привести все названия столбцов к нему.

---

## Проверка ошибок в данных и их предобработка


### Названия, или метки, столбцов датафрейма

In [5]:
#Выведем названия всех столбцов датафрейма 
print(f'Названия столбцов до корректировки: {df.columns}')

Названия столбцов до корректировки: Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')


In [6]:
#Приведём названия всех столбцов датафрейма к стилю snake_case
df.columns = df.columns.str.lower().str.replace(' ', '_')
print(f'Названия столбцов после приведения к стилю snake_case: {df.columns}')

Названия столбцов после приведения к стилю snake_case: Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


Наименования всех столбцов датафрейма приведены к стилю snake_case для соблюдения "хорошего стиля" Python и удобства дальнейшей работы с данными.

### Типы данных

После предварительного обзора датафрейма мы выяснили, что в 10-ти из 11-ти столбцов  типы данных нужно преобразовать. Ошибки в присвоении типов столбцам возникли во время загрузки датасета в pandas из csv-файла и связаны они с тем, что в столбцах, в которых по описанию должны быть числовые значения, помимо них встречаются пропуски и строки.

1. В столбце `year_of_release` нам нужно преобразовать тип с `float64` на `int16`. Но мы уже знаем, что в этом столбце присутствуют пропуски и сначала нужно обработать их. Это мы сделаем в слудующем подразделе, а затем произведём преобразование типа в этом столбце.

2. В столбцах `na_sales`, `other_sales` и `critic_score` снизим разрядность типа на `float32` для оптимизации ресурсов:

In [7]:
#Преобразуем тип в столбцах na_sales, other_sales и critic_score на float32 и выведем тип для проверки результата операции
df[['na_sales', 'other_sales','critic_score']] = df[['na_sales', 'other_sales','critic_score']].apply(pd.to_numeric, 
                                                                                                      downcast='float', 
                                                                                                      errors='coerce')
print(f"Тип в столбце na_sales: {df['na_sales'].dtypes}")
print(f"Тип в столбце other_sales: {df['other_sales'].dtypes}")
print(f"Тип в столбце critic_score: {df['critic_score'].dtypes}")

Тип в столбце na_sales: float32
Тип в столбце other_sales: float32
Тип в столбце critic_score: float32


Преобразование типа в трёх столбцах прошло успешно.

3. В столбцах `platform`, `genre` и `rating` преобразуем тип данных на `category`:

In [8]:
#Преобразовываем тип данных в столбцах platform, genre и rating на category
df[['platform', 'genre', 'rating']] = df[['platform', 'genre', 'rating']].astype('category')
print(df[['platform', 'genre', 'rating']].dtypes)

platform    category
genre       category
rating      category
dtype: object


Преобразование типа прошло успешно.

4. В столбцах `eu_sales`, `jp_sales` и `user_score` нам нужно преобразовать тип данных с object на float32:

Так как при загрузке датасета из файла в pandas этим столбцам присвоился тип object, значит мы можем предположить, что в них помимо цифр встречаются строки. Чтобы преобразование прошло без ошибок используем параметр errors='coerce', чтобы текст автоматически преобразовывался в пропуски.

In [9]:
#Преобразуем тип в столбцах eu_sales, jp_sales и user_score на float32 и выведем тип для проверки результата операции
df[['eu_sales', 'jp_sales','user_score']] = df[['eu_sales', 'jp_sales','user_score']].apply(pd.to_numeric, 
                                                                                                      downcast='float', 
                                                                                                      errors='coerce')
print(f"Тип в столбце eu_sales: {df['eu_sales'].dtypes}")
print(f"Тип в столбце jp_sales: {df['jp_sales'].dtypes}")
print(f"Тип в столбце user_score: {df['user_score'].dtypes}")

Тип в столбце eu_sales: float32
Тип в столбце jp_sales: float32
Тип в столбце user_score: float32


Преобразование типа в трёх столбцах теперь прошло успешно. Мы преобразовали типы в 9-ти столбцах. И ещё в одном `year_of_release` нам перед преобразованием типа нужно сначала обработать пропуски (сделаем это в следующем подразделе).

### Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [10]:
#Выведем количество пропусков в каждом столбце в порядке убывания значения
print(df.isna().sum().sort_values(ascending=False))

user_score         9268
critic_score       8714
rating             6871
year_of_release     275
eu_sales              6
jp_sales              4
name                  2
genre                 2
platform              0
na_sales              0
other_sales           0
dtype: int64


In [11]:
#Выведем долю пропусков в каждом столбце в порядке убывания значения
print(df.isna().mean().sort_values(ascending=False))

user_score         0.546591
critic_score       0.513918
rating             0.405225
year_of_release    0.016218
eu_sales           0.000354
jp_sales           0.000236
name               0.000118
genre              0.000118
platform           0.000000
na_sales           0.000000
other_sales        0.000000
dtype: float64


Пропуски теперь присутствуют в 8-ми из 11-ти столбцов - до преобразования данных они были в 6-ти, но вследствие преобразования типа данных столбцах `eu_sales` и `jp_sales` с `object` на `float32` мы увеличили количество случаев пропусков в столбцах. 

Чтобы проанализировать возможные причины возникновения пропусков в наших данных, попробуем выявить взаимосвязи их возникновения - посмотрим, как пропуски распределяются по годам выпуска игр:

In [12]:
#Выведем долю пропусков в каждом столбце, сгрупировав данные по годам выпуска
def df_missing_share(df):
    # Группируем данные по столбцу year_of_release
    grouped = df.groupby('year_of_release')
    
    # Вычисляем долю пропусков для каждой группы
    missing_share = grouped.apply(lambda x: x.isna().mean())
    
    return missing_share


df_missing_share(df)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
year_of_release,,,,,,,,,,,
1980.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1981.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1982.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1983.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1984.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1985.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.928571,0.928571,0.928571
1986.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1987.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.000000,1.000000,1.000000
1988.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.933333,0.933333,0.933333


В столбцах с названием игр и жанрами все имеющиеся пропуски сконцентрированы в один год - 1993. Мы можем посмотреть, связаны ли они как-то между собой:

In [13]:
#Выведем строки с пропусками в столбцах name и genre
df.loc[(df['year_of_release'] == 1993.0) & ((df['name'].isna()) | (df['genre'].isna()))]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
661,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14439,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Мы видим, что пропуски в в столбцах `name` и `genre` относятся к одним и тем же записям. То есть распределены не полностью случайно - есть взаимосвязь по этим двум столбцам. Обе записи частично заполнены и не идут подряд. Какой-то явной видимой причины их возникновения по имеющейся у нас в распоряжении информации нельзя определить. Причиной могут быть технические сбои при сборе или выгрузке данных. По количеству и доле эти пропуски единичные, и в дальнейшем анализе в рамках проекта эти столбцы не используются, поэтому пропуски в них можно оставить.

В столбцах `eu_sales`, `jp_sales` пропусков тоже очень мало (6 и 4 шт. соответственно). Их распределение по годам выглядит случайным, кроме 2008 года. Посмотрим на строки с пропусками в этих столбцах в 2008 году:

In [14]:
#Выведем строки с пропусками в столбцах eu_sales и jp_sales
df.loc[(df['year_of_release'] == 2008.0) & ((df['eu_sales'].isna()) | (df['jp_sales'].isna()))]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
446,Rhythm Heaven,DS,2008.0,Misc,0.55,NaN,1.93,0.13,83.0,9.0,E
467,Saints Row 2,X360,2008.0,Action,1.94,0.79,NaN,0.28,81.0,8.1,M


Пропуски в этих двух столбцах относятся к разным записям. Соответственно распределение всех пропусков в этих двух столбцах выглядит случайным и определить какую-то взаимосвязь между ними на основании имеющейся у нас информации не представляется возможным. Причинами возникновения этих пропусков могут быть технические или программные ошибки или человеческий фактор. Согласно заданию заполним пропуски в этих столбцах средним с группировкой по году выпуска и платформе:

In [15]:
#С помощью пользовательской функции заполним пропуски в столбце eu_sales средним с группировкой по году выпуска и платформе

def mean_group_sales(row):
    """
Функция принимает на вход строки датафрейма.
В переменной group мы сохраняем датафрейм, сгруппированный по году выпуска и платформе.
Если в перебираемой ячейке есть пропуск - он заменяется на среднее значение внутри группы.
Если пропуска нет - остаётся исходное значение ячейки.
    """
    if pd.isna(row['eu_sales']):
        group = df[(df['platform'] == row['platform']) & 
                   (df['year_of_release'] == row['year_of_release'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']

df['eu_sales'] = df.apply(mean_group_sales, axis=1)
print(f"Количество пропусков в столбце eu_sales после замены: {df['eu_sales'].isna().sum()}")

Количество пропусков в столбце eu_sales после замены: 0


In [16]:
#Аналогичную функцию используем для заполнения пропусков в столбце jp_sales
def mean_group_sales(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['platform'] == row['platform']) & 
                   (df['year_of_release'] == row['year_of_release'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']

df['jp_sales'] = df.apply(mean_group_sales, axis=1)
print(f"Количество пропусков в столбце jp_sales после замены: {df['jp_sales'].isna().sum()}")

Количество пропусков в столбце jp_sales после замены: 0


Мы успешно заполнили пропуски в столбцах `eu_sales` и `jp_sales`.

В столбце с годом выпуска игры `year_of_release` пропусков 16%. Доля пропусков уже заметная. Удалять такое количество строк нельзя - данные в них важны для анализа по другим столбцам. Но и заполнить их, кроме как вручную гугля год выпуска каждой игры, нет оптимального способа, а это будет слишком затратно по времени - это 275 строк с разными названиями игр. Просто проигнорировать эти пропуски тоже не получится - нам нужно преобразовать тип данных в этом столбце на `int16`, чтобы в дальнейшем была возможность сделать выборку игр по годам выпуска. Заменим пропуски на значением-индикатором 0 - так нам удастся преобразовать тип и не потерять данные этих строк при анализе по другим столбцам. Предварительно проверим диапазон значений, встречающихся в столбце.

Попытаемся проанализировать причины возникновения пропусков в столбце `year_of_release`. Для этого рассмотрим распределение пропусков по группам-платформам и группам-жанрам:

In [17]:
def df_missing_share(df):
    # Группируем данные по столбцу 
    grouped = df.groupby('platform')
    
    # Вычисляем долю пропусков для каждой группы
    missing_share = grouped.apply(lambda x: x.isna().mean())
    
    return missing_share

df_missing_share(df)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
platform,,,,,,,,,,,
2600,0.000000,0.0,0.125926,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000
3DO,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000
3DS,0.000000,0.0,0.015094,0.000000,0.0,0.0,0.0,0.0,0.681132,0.669811,0.562264
DC,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.730769,0.730769,0.730769
DS,0.000000,0.0,0.013780,0.000000,0.0,0.0,0.0,0.0,0.667432,0.760220,0.409738
GB,0.000000,0.0,0.010204,0.000000,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000
GBA,0.000000,0.0,0.013142,0.000000,0.0,0.0,0.0,0.0,0.470729,0.691756,0.367981
GC,0.000000,0.0,0.024867,0.000000,0.0,0.0,0.0,0.0,0.195382,0.342806,0.158082
GEN,0.068966,0.0,0.000000,0.068966,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000


In [18]:
def df_missing_share(df):
    # Группируем данные по столбцу 
    grouped = df.groupby('genre')
    
    # Вычисляем долю пропусков для каждой группы
    missing_share = grouped.apply(lambda x: x.isna().mean())
    
    return missing_share

df_missing_share(df)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
genre,,,,,,,,,,,
ACTION,0.0,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.384615,0.384615,0.307692
ADVENTURE,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000
Action,0.0,0.0,0.018796,0.0,0.0,0.0,0.0,0.0,0.442291,0.459031,0.352423
Adventure,0.0,0.0,0.007582,0.0,0.0,0.0,0.0,0.0,0.752085,0.770281,0.658074
FIGHTING,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.500000,0.333333
Fighting,0.0,0.0,0.014019,0.0,0.0,0.0,0.0,0.0,0.519860,0.530374,0.483645
MISC,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.666667,1.000000,0.666667
Misc,0.0,0.0,0.016393,0.0,0.0,0.0,0.0,0.0,0.699830,0.747880,0.495195
PLATFORM,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.333333


Распределение пропусков в столбце `year_of_release` в обоих случаях выглядит случайным - взаимосвязей с другими данными или определённой логики не прослеживается. Причина появления этих пропусков, вероятно, техническая или программная.

И в трёх столбцах доля пропущены значения в половине случаев:  `user_score` (54,7%), `critic_score` (51,4%) и `rating` (40,5%). Эти строки мы никак не можем отбросить - их слишком много, а данные столбцы участвуют в дальнейшем анализе.

Если вернуться к распределению пропусков в столбцах датафрейма по годам выпуска игр, мы увидим, что пропуски в этих трёх столбцах составляют почти 100% с 1980 по 1999 год включительно. Видимо, это связано с тем, что интернет в эти годы был ещё не настолько развит и недоступен пользователям, и собирать оценки пользователей и критиков просто не было механизма. Согласно Яндекс.Браузеру, ESRB (Entertainment Software Rating Board) начала присваивать рейтинги компьютерным играм только в 1994 году. То есть в этом случае механизм тоже ещё какое-то время налаживался. Судя по нашим данным датасета - примерно до 2000 года. Но и далее по максимальный год в наших данных (2016) система сбора оценок и присвоения рейтингов развивалась, но не достигла совершенства.

В случае оценок пользователей и критиков заполнение средним или медианой даже с группировкой будет слишком неточным решением, ведь рейтинги − вещь очень индивидуальная. Исказить данные нам бы не хотелось. Если здесь заполним средним, то рискуем получить искаженные пропорции игр по рейтингам. В нашем случае хорошим вариантом будет заполнить нереалистичным значением−заглушкой типа «-1», по которому затем можно будет легко отфильтровать данные.

Пропусков в столбце `rating` с рейтингом ESRB 40,5%, значит мы не можем отбросить эти строки, их слишком много. Но данные этого столбца не понадобятся нам в дальнейшем анализе игр, поэтому мы можем их оставить как есть.

1. Перед заменой посмотрим диапазон значений, встречающихся в столбце `year_of_release`, чтобы убедиться, что значение-индикатор 0 не встречается в нём:

In [19]:
#Выведем основные статистики столбца year_of_release для оценки его содержимого
df['year_of_release'].describe()

count    16681.000000
mean      2006.485522
std          5.873102
min       1980.000000
25%       2003.000000
50%       2007.000000
75%       2010.000000
max       2016.000000
Name: year_of_release, dtype: float64

Значение 0 в столбце не встречается - используем его для замены пропусков и затем преобразуем тип данных в этом столбце на `int16`:

In [20]:
#Заменяем пропуски в столбце year_of_release на значение-индикатор 0 и затем преобразуем тип данных столбца на int16
df['year_of_release']=df['year_of_release'].fillna(0)
df['year_of_release'] = pd.to_numeric(df['year_of_release'], downcast='integer')
print(df['year_of_release'].value_counts())
print(f"Тип данных столбца year_of_release после преобразования: {df['year_of_release'].dtypes}")

2009    1450
2008    1445
2010    1279
2007    1218
2011    1149
2006    1020
2005     950
2002     839
2003     789
2004     771
2012     670
2015     612
2014     584
2013     552
2016     507
2001     491
1998     384
2000     357
1999     341
1997     293
0        275
1996     267
1995     220
1994     121
1993      62
1981      46
1992      43
1991      42
1982      37
1986      22
1983      18
1987      17
1989      17
1990      16
1988      15
1984      14
1985      14
1980       9
Name: year_of_release, dtype: int64
Тип данных столбца year_of_release после преобразования: int16


Обработка пропусков и последующее преобразование типа прошли успешно.

2. Нам нужно попытаться найти оптимальный способ заполнить пропуски в столбце `user_score` - оценка пользователей (54,7% пропущенных значений). Далее в исследовании нам на основе этих оценок нужно будет категоризовать игры.

Чтобы не искажать имеющиеся в распоряжении данные по оценкам пользователей, заполним пропуски в этом столбце значением-индикатором "-1", которому не может быть равна реальная оценка и по которому затем можно будет легко отфильтровать данные:

In [21]:
#Заменяем пропуски в столбце user_score на значение-индикатор -1
df['user_score']=df['user_score'].fillna(-1)

#Выведем основные статистики столбца user_score для оценки его заполненных значений
print(df['user_score'].describe())

print(f"Количество пропусков в столбце user_score после заполнения пропусков: {df['user_score'].isna().sum()}")

count    16956.000000
mean         2.682862
std          4.167944
min         -1.000000
25%         -1.000000
50%         -1.000000
75%          7.300000
max          9.700000
Name: user_score, dtype: float64
Количество пропусков в столбце user_score после заполнения пропусков: 0


Теперь в столбце `user_score` нет пропусков.

3. Теперь нужно  обработать пропуски в столбце `critic_score` с оценками критиков - их в исходном датасете было 51,4%. На основе этих оценок позже нам нужно будет категоризовать игры, поэтому проигнорировать пропуски мы не можем. Удалить - тем более, ввиду их большого количества - это половина данных.

С оценками критиков поступим аналогично оценкам игроков. Чтобы не искажать имеющиеся в распоряжении данные, заполним пропуски в этом столбце значением-индикатором "-1":

In [22]:
#Выведем количество пропусков в столбце critic_score после удаления строк с пропусками в столбце user_score и 
#перед обработкой пропусков в этом столбце
print(f"Количество пропусков в столбце critic_score: {df['critic_score'].isna().sum()}")

Количество пропусков в столбце critic_score: 8714


В стоблце `critic_score` перед обработкой пропусков их присутствует 8714.

In [23]:
#Заменяем пропуски в столбце critic_score на значение-индикатор -1
df['critic_score']=df['critic_score'].fillna(-1)

#Выведем основные статистики столбца critic_score для оценки его заполненных значений
print(df['critic_score'].describe())

print(f"Количество пропусков в столбце critic_score после заполнения пропусков: {df['critic_score'].isna().sum()}")

count    16956.000000
mean        32.990093
std         36.277740
min         -1.000000
25%         -1.000000
50%         -1.000000
75%         70.000000
max         98.000000
Name: critic_score, dtype: float64
Количество пропусков в столбце critic_score после заполнения пропусков: 0


Теперь в столбце `critic_score` не осталось пропусков.

4. Теперь нужно принять решение по пропускам в столбце `rating` с рейтингом ESRB. Их тут 40,5%, значит мы не можем отбросить эти строки, их слишком много. Но данные этого столбца не понадобятся нам в дальнейшем анализе игр, поэтому мы можем игнорировать пропуски в этом столбце.

Мы обработали все столбцы, в которых были пропуски: в исходном датафрейме их было 6, после преобразования типов данных добавились ещё 2 столбца. По итогам обработки мы заполнили пропуски в 5-ти столбцах и в 3-х случаях приняли решение оставить пропуски, так как они не помешают дальнейшему исследованию.

### Явные и неявные дубликаты в данных

1. Изучим все уникальные значения в столбцах с категориальными данными - названиями жанров, платформ, рейтинга ESRB - на предмет неявных дубликатов, связанных с опечатками или разным стилем написания. При необходимости нормализуем значения в столбцах:

In [24]:
#Выведем список уникальных значений в столбце genre
df['genre'].value_counts()

Action          3405
Sports          2367
Misc            1769
Role-Playing    1510
Shooter         1341
Adventure       1319
Racing          1267
Platform         901
Simulation       882
Fighting         856
Strategy         690
Puzzle           588
ACTION            13
SPORTS             8
ROLE-PLAYING       6
FIGHTING           6
RACING             6
SHOOTER            5
ADVENTURE          4
PLATFORM           3
MISC               3
PUZZLE             2
SIMULATION         2
STRATEGY           1
Name: genre, dtype: int64

По выведенным значениям мы видим, что есть одинаковые значения, но с разным регистром. Приведём все названия жанров к нижнему регистру и снова выведем список уникальных вариантов значений:

In [25]:
#Приведём все названия жанров к нижнему регистру и выведем новый список уникальных значений и их количество
df['genre'] = df['genre'].str.lower()
print(f"Уникальные значения в столбце genre: {df['genre'].unique()}")
print(f"Количество жанров: {df['genre'].nunique()}")

Уникальные значения в столбце genre: ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy' nan]
Количество жанров: 12


После приведения к одному регистру количество уникальных жанров стало равно 12-ти и по их списку мы видим, что теперь они действительно уникальные.

In [26]:
#Выведем список уникальных значений в столбце platform
df['platform'].value_counts()

PS2     2189
DS      2177
PS3     1355
Wii     1340
X360    1281
PSP     1229
PS      1215
PC       990
XB       839
GBA      837
GC       563
3DS      530
PSV      435
PS4      395
N64      323
XOne     251
SNES     241
SAT      174
WiiU     147
2600     135
NES      100
GB        98
DC        52
GEN       29
NG        12
SCD        6
WS         6
3DO        3
TG16       2
PCFX       1
GG         1
Name: platform, dtype: int64

Похоже, в столбце с названиями платформ нет дублирующихся значений, связанных с опечатками или разным способом написания. Оставим значения в этом столбце как есть.

In [27]:
#Выведем список уникальных значений в столбце rating
df['rating'].unique()

['E', NaN, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (8, object): ['E', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']

В столбце с рейтингом ESRB встречается 7 вариантов уникальных значений, которые соответствует списку, приведённому в описании данных, и есть одно значение, которого в списке нет: "K-A". Обратимся к Яндекс.Браузеру за выяснением, что эта аббревиатура можнт значить: "K-A (Kids to Adults) — это историческое обозначение рейтинга ESRB, которое использовалось до 1998 года. Позже оно было заменено на более привычный сейчас рейтинг E (Everyone — «Для всех»)". Таким образом, мы можем заменить значение 'K-A' на 'E', чтобы привести список в соответствие с описанием данных и чтобы не провоцировать разночтения одного и того же значения при последующем использовании этих данных:

In [28]:
#Заменим значение 'K-A' на 'E' и выведем новый список уникальных значений и их количество
df['rating'] = df['rating'].replace('K-A', 'E')
print(df['rating'].unique())

['E', NaN, 'M', 'T', 'E10+', 'AO', 'EC', 'RP']
Categories (7, object): ['E', 'M', 'T', 'E10+', 'AO', 'EC', 'RP']


Мы обработали неявные дубликаты в столбцах с категориальными данными.

2. Теперь проверим данные на наличие явных дубликатов:

In [29]:
#Выведем количество полностью дублирующих имеющиеся записей
df.duplicated().sum()

241

В датафрейме на данный момент найдено 241 штука полностью дублирующих имеющиеся записей. Удалим их, поскольку они будут вносить ошибки в дальнейший анализ данных этого датасета:

In [30]:
#Выведем количество строк исходного датафрейма, удалим явные дубликаты (210 строк) и выведем для проверки количество строк
#очищенного датафрейма
print(f"Количество строк в исходном датафрейме: {df.shape[0]}")
df_cleaned = df.drop_duplicates()
print(f"Количество строк в датафрейме после удаления явных дубликатов: {df_cleaned.shape[0]}")

Количество строк в исходном датафрейме: 16956
Количество строк в датафрейме после удаления явных дубликатов: 16715


Удалена ровно 241 строка с явными дубликатами, очищенный от дубликатов датафрейм сохранён в переменную `df_cleaned`.

3. Посчитаем количество удалённых в результате подготовки данных строк исходного датафрейма `df` относительно `df_cleaned` в абсолютном и относительном значениях:

In [31]:
#Рассчитаем количество удалённых строк и их долю
print(16956 - df_cleaned.shape[0])
print(((16956 - df_cleaned.shape[0]) / 16956) * 100)

241
1.421325784383109


В результате подготовки данных к анализу удалено 1,42% строк исходного датафрейма.

В результате проведения подготовки данных к анализу была проведена нормализация названий столбцов, преобразование типов данных, обработка пропусков (на этом этапе мы строки не удаляли), обработка неявных дубликатов и очистка данных от явных дубликатов. Удалённые строки (241 штука) составили 1,42% строк исходного датасета.

---

## Фильтрация данных

Для этого и дальнейших разделов создадим копию датафрейма, чтобы вносить изменения в копию, не боясь испортить данные.

In [32]:
df_actual = df_cleaned.copy()


Отберём данные за период с 2000 по 2013 год включительно, сохраним новый срез данных в отдельном датафрейме `df_actual`:

In [33]:
#Присвоим переменной df_actual срез из очищенного датафрейма за период с 2000 по 2013 год включительно
df_actual = df_actual.loc[(df_actual['year_of_release'] >= 2000) & (df_actual['year_of_release'] <= 2013)]
#Для проверки результата выведем список уникальных значений года в срезе, их количество и сколько строк в датафрейме,
#с которым мы будем дальше работать
print(f"Уникальные значения в столбце year_of_release после фильтрации: {df_actual['year_of_release'].unique()}")
print(f"Количество лет после фильтрации: {df_actual['year_of_release'].nunique()}")
print(f"Количество строк в отфильтрованном датафрейме: {df_actual.shape[0]}")

Уникальные значения в столбце year_of_release после фильтрации: [2006 2008 2009 2005 2007 2010 2013 2004 2002 2001 2011 2012 2003 2000]
Количество лет после фильтрации: 14
Количество строк в отфильтрованном датафрейме: 12781


В новый датафрейм `df_actual` сохранен срез очищенных данных за период с 2000 по 2013 год включительно. В него вошло 12797 строк.

---

## Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

1. Проведём категоризацию данных - разделим все игры по оценкам пользователей и выделим такие категории: 
- высокая оценка (от 8 до 10 включительно),
- средняя оценка (от 3 до 8, не включая правую границу интервала) и
- низкая оценка (от 0 до 3, не включая правую границу интервала).

In [34]:
#Добавим в датафрейм новый столбец categories_users с категориями на основе оценок пользователей
df_actual['categories_users'] = pd.cut(df_actual['user_score'], bins=[0, 3, 8, 10], labels=['низкая оценка',
                                                                                            'средняя оценка', 
                                                                                            'высокая оценка'],
                                                                                            right=False)
#Для проверки результата выведем первые 20 строк нового столбца
df_actual[['user_score', 'categories_users']].head(20)

,user_score,categories_users
0,8.0,высокая оценка
2,8.3,высокая оценка
3,8.0,высокая оценка
6,8.5,высокая оценка
7,6.6,средняя оценка
8,8.4,высокая оценка
10,-1.0,NaN
11,8.6,высокая оценка
13,7.7,средняя оценка
14,6.3,средняя оценка


Категоризация на основе оценок пользователей выполнена. Границы категорий совпадают с поставленными в задаче.

2. Разделим все игры по оценкам критиков и выделим такие категории:
   - высокая оценка (от 80 до 100 включительно),
   - средняя оценка (от 30 до 80, не включая правую границу интервала) и
   - низкая оценка (от 0 до 30, не включая правую границу интервала).

In [35]:
#Добавим в датафрейм новый столбец categories_critics с категориями на основе оценок критиков
df_actual['categories_critics'] = pd.cut(df_actual['critic_score'], bins=[0, 30, 80, 100], labels=['низкая оценка', 
                                                                                                   'средняя оценка',
                                                                                                   'высокая оценка'],
                                                                                                    right=False)
#Для проверки результата выведем первые 20 строк нового столбца
df_actual[['critic_score', 'categories_critics']].head(20)

,critic_score,categories_critics
0,76.0,средняя оценка
2,82.0,высокая оценка
3,80.0,высокая оценка
6,89.0,высокая оценка
7,58.0,средняя оценка
8,87.0,высокая оценка
10,-1.0,NaN
11,91.0,высокая оценка
13,80.0,высокая оценка
14,61.0,средняя оценка


Категоризация на основе оценок критиков выполнена. Границы категорий совпадают с поставленными в задаче.

3. Проверим результат после категоризации данных - сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории:

In [36]:
#Сгруппируем данные по двум новым столбцам с категориями оценок пользователей и критиков и посчитаем количество игр,
#вошедших в каждую категорию
grouped_data = df_actual.groupby(['categories_users', 'categories_critics'], as_index=False)['name'].count()
print(grouped_data)

  categories_users categories_critics  name
0    низкая оценка      низкая оценка    17
1    низкая оценка     средняя оценка    77
2    низкая оценка     высокая оценка     1
3   средняя оценка      низкая оценка    30
4   средняя оценка     средняя оценка  3157
5   средняя оценка     высокая оценка   641
6   высокая оценка      низкая оценка     1
7   высокая оценка     средняя оценка  1167
8   высокая оценка     высокая оценка  1017


Подавляющее большинство выпущенных в этот период игр (9682 из 12797) получили одновременно среднюю оценку и от пользователей, и от критиков. Высокой оценки и от пользователей, и от критиков удостоились 1017 игр. И 17 игр получили низкий рейтинг и от критиков, и от зрителей:

In [37]:
#Выведем список 17-ти игр, получивших низкую оценку и от критиков, и от игроков
df_actual['name'].loc[(df_actual['categories_users'] == 'низкая оценка') & (df_actual['categories_critics'] == 'низкая оценка')]

1057                         Deal or No Deal
7556           Mortal Kombat: Special Forces
8602     Leisure Suit Larry: Box Office Bust
9464                           Rogue Warrior
9480                 Hulk Hogan's Main Event
10499                          Rogue Warrior
10805    Leisure Suit Larry: Box Office Bust
10872                 Dragon Ball: Evolution
11359                Jumper: Griffin's Story
11595                         Ninjabread Man
12131                          Homie Rollerz
12296                              Anubis II
12997                           Ride to Hell
13296                       Jenga World Tour
13877                           Ride to Hell
15562                          Balls of Fury
15994                            Pulse Racer
Name: name, dtype: object

4. Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период:

In [38]:
#Выводим "голову" из 7 строк списка уникальных названий платформ в порядке убавыния относящихся к ним строк (игр)
df_actual['platform'].value_counts().head(7)

PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: platform, dtype: int64

Больше всего игр в исследуемый период вышло на платформе PlayStation 2. Недалеко от неё (всего на 5 игр) отстаёт Nintendo DS. Следующие в списке платформы уже отстают на порядка тысячи игр, выпущенных в этот отрезок лет (с 2000 по 2013 год).

---

## Итоговый вывод

В рамках данного проекта мы изучали данные о развитии игровой индустрии в начале XXI века. С этой целью мы выполнили следующие задачи:
- загрузили данные из датасета, познакомились с их характеристиками: объёмом, описанием и оформлением после загрузки в датафрейм, с типами данных, которые были присвоены столбцам после загрузки в датафрейм, отметили присутствие и долю пропусков в данных - в некоторых столбцах их было порядка половины общего объёма данных;
- затем мы подготовили данные к последующему анализу: откорректировали названия столбцов с соответствии с основным стилем Python, преобразовали типы данных и тем самым оптимизировали ресурсы системы и сделали данные удобными для дальнейшей работы с ними, обработали пропуски, обработали неявные дубликаты, удалили явные дубликаты (241, 1,42%);
- в основных разделах работы с предобработанными данными мы отфильтровали весь объём данных по требующемуся для анализа временному периоду (с 2000 по 2013 год включительно), затем мы категоризовали игры по оценкам пользователей и критиков(ввели в датафрейм новые столбцы `categories_users` и `categories_critics`), сделали группировку данных по этим новым категориям и посчитали количество игр в каждой категории, также мы выявили топ-7 игровых платформ по количеству игр, выпущенных за изучаемый период.